<a href="https://colab.research.google.com/github/Swathivijayalakshmi/codemind-java/blob/main/Copy_of_IBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns',None)
burnoutDf=pd.read_csv('/content/train.csv')
burnoutDf

In [ ]:
burnoutDf.shape

In [ ]:
burnoutDf.info()

In [ ]:
burnoutDf.head

In [ ]:
burnoutDf.isna().sum()

In [ ]:
burnoutDf.describe

In [ ]:
for i,col in enumerate(burnoutDf.columns):
  print(f"\n\n{burnoutDf[col].unique()}")
  print(f"\n{burnoutDf[col].value_counts()}\n\n")

In [ ]:
burnoutDf=burnoutDf.drop(['Employee ID'],axis=1)

In [ ]:
intFloatburnoutDf=burnoutDf.select_dtypes([np.int,np.float])
for i,col in enumerate(intFloatburnoutDf.columns):
  if(intFloatburnoutDf[col].skew()>=0.1):
    print("\n",col,"feature is positively skewed and value is:",intFloatburnoutDf[col].skew())
  elif(intFloatburnoutDf[col].skew()<=-0.1):
    print("\n",col,"featute is negatively skewed and value is:",intFloatburnoutDf[col].skew())
  else:
      print("\n",col,"feature is normally Distributed and value is:",intFloatburnoutDf[col].skew())

In [ ]:
burnoutDf['Resource Allocation'].fillna(burnoutDf['Resource Allocation'].mean(),inplace=True)
burnoutDf['Mental Fatigue Score'].fillna(burnoutDf['Mental Fatigue Score'].mean(),inplace=True)
burnoutDf['Burn Rate'].fillna(burnoutDf['Burn Rate'].mean(),inplace=True)

In [ ]:
burnoutDf.isna().sum()

In [ ]:
burnoutDf.corr()

In [ ]:
Corr=burnoutDf.corr()
sns.set(rc={'figure.figsize':(14,12)})
fig=px.imshow(Corr,text_auto=True,aspect="auto")
fig.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(x="WFH Setup Available",data=burnoutDf,palette="dark:salmon_r")
plt.title("Plot Distribution of WFH_Setup_Available")
plt.show()

In [ ]:
plt.figure(figsize=(10,18))
sns.countplot(x="Gender",data=burnoutDf,palette="magma")
plt.title("Plot Distribution of Gender")
plt.show()

In [ ]:
plt.figure(figsize=(10,18))
sns.countplot(x="Company Type",data=burnoutDf,palette="Spectral")
plt.title("Plot Distribution of company Type")
plt.show()

In [ ]:
burn_st=burnoutDf.loc[:,'Date of Joining':'Burn Rate']
burn_st=burn_st.select_dtypes([int,float])
for i,col in enumerate(burn_st.columns):
  fig=px.histogram(burn_st,x=col,title="Plot Distribution of "+col,color_discrete_sequence=['indianred'])
  fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
fig=px.line(burnoutDf,y="Burn Rate",color="Designation",title="Burn rate on the basis of Designation",color_discrete_sequence=px.colors.qualitative.Pastel1)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
fig=px.line(burnoutDf,y="Mental Fatigue Score",color="Designation",title="Mental fatigue vs Designation",color_discrete_sequence=px.colors.qualitative.Pastel1)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
sns.relplot(
    data=burnoutDf,x="Designation",y="Mental Fatigue Score",col="Company Type",
    hue="Company Type",size="Burn Rate",style="Gender",
    palette=["g","r"],sizes=(50,200)
)

In [ ]:
from sklearn import preprocessing
Label_encode=preprocessing.LabelEncoder()

In [ ]:
burnoutDf['GenderLabel']=Label_encode.fit_transform(burnoutDf['Gender'].values)
burnoutDf['Company_TypeLabel']=Label_encode.fit_transform(burnoutDf['Company Type'].values)
burnoutDf['WFH_Setup_AvailableLabel']=Label_encode.fit_transform(burnoutDf['WFH Setup Available'].values)

In [ ]:
gn=burnoutDf.groupby('Gender')
gn=gn['GenderLabel']
gn.first()

In [ ]:
ct=burnoutDf.groupby('Company Type')
ct=ct['Company_TypeLabel']
ct.first()

In [ ]:
wsa=burnoutDf.groupby('WFH Setup Available')
wsa=wsa['WFH_Setup_AvailableLabel']
wsa.first()

In [ ]:
burnoutDf.tail(10)

In [ ]:
Columns=['Designation','Resource Allocation','Mental Fatigue Score',
         'GenderLabel','Company_TypeLabel','WFH_Setup_AvailableLabel']
x=burnoutDf[Columns]
y=burnoutDf['Burn Rate']

In [ ]:
print(x)

In [ ]:
print(y)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(0.95)
x_pca=pca.fit_transform(x)
print("PCA shape of x is:",x_pca.shape,"and original shape is:",x.shape)
print("% of importance of selected features is:",pca.explained_variance_ratio_)
print("The number of features selected through PCA is:",pca.n_components_)

In [ ]:
from sklearn.model_selection import train_test_split
x_train_pca,x_test,y_train,y_test=train_test_split(x_pca,y,test_size=0.25,random_state=10)

In [ ]:
print(x_pca.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor()
rf_model.fit(x_train_pca,y_train)
train_pred_rf=rf_model.predict(x_train_pca)
train_r2=r2_score(y_train,train_pred_rf)
test_pred_rf=rf_model.predict(x_test)
test_r2=r2_score(y_test,test_pred_rf)
print("Accuracy score of tarin data:"+str(round(100*train_r2,4))+"%")
print("Accuracy score of test data:"+str(round(100*test_r2,4))+"%")